In [1]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.forecasting.stl import STLForecast
from sktime.performance_metrics.forecasting import mean_absolute_scaled_error

from tqdm import tqdm
from statsmodels.tsa.seasonal import STL

import sys 
sys.path.insert(1, "../")
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import ast

import dask.dataframe as dd
from dask.dataframe import from_pandas
import json
from workloads.util import use_results, use_dataset, join_queries_features

%load_ext autoreload
%autoreload 2

ImportError: cannot import name '_centered' from 'scipy.signal.signaltools' (/data/wooders/anaconda3/lib/python3.8/site-packages/scipy/signal/signaltools.py)

In [3]:
experiment = "yahoo/A1"

results_dir = use_results(experiment)
dataset_dir = use_dataset(experiment, download=False)
print(results_dir)
print(dataset_dir)

/data/wooders/ralf-vldb//datasets/yahoo/A1
/data/wooders/ralf-vldb//results/yahoo/A1
/data/wooders/ralf-vldb//datasets/yahoo/A1


In [4]:
init_forecast = json.load(open(f"{dataset_dir}/init_forecast.json"))

In [5]:
result_name = "results_workers_1_rr_window_48_slide_1"

result_file = f"{result_name}.csv"
timestamp_file = f"{result_name}_timestamps.csv"
result_df = pd.read_csv(f"{results_dir}/{result_file}")
#result_df['trend'] = result_df['trend'].dropna().apply(ast.literal_eval)
timestamp_df = pd.read_csv(f"{results_dir}/{timestamp_file}")

In [6]:
timestamp_df

,timestamp,processing_time
0,0,1.647463e+09
1,1,1.647463e+09
2,2,1.647463e+09
3,3,1.647463e+09
4,4,1.647463e+09
...,...,...
1415,1415,1.647463e+09
1416,1416,1.647463e+09
1417,1417,1.647463e+09
1418,1418,1.647463e+09


In [7]:
result_df.key_id.value_counts()

1     4
2     3
3     3
4     3
5     3
6     3
7     3
8     3
9     2
10    2
11    2
12    2
13    2
14    2
Name: key_id, dtype: int64

In [ ]:
keys = range(1, 15, 1)

pred_len = 700 - window_size
predictions = {key: [None]*pred_len for key in keys}
rows = []
for ts in range(window_size, 1, 700): 
    query_time = timestamp_df.iloc[ts].processing_time

    for key in keys: 
        model_ts = result_df[(result_df["key_id"] == key) & (result_df["processing_time"] <= query_time)].timestamp.max() 
        forecast = result_df[(result_df["key_id"] == key) & (result_df["timestamp"] == model_ts)].forecast

        if model_ts is None: 
            forecast = init_forecast[key]
            model_ts = 0

        rows.append({
            "key": key, 
            "model_ts": model_ts, 
            "ts": ts,
            "pred_value": forecast[ts - model_ts], 
            "value": data[key][ts]

        })

df = pd.DataFrame(rows)

In [ ]:
keys = range(1, 15, 1)
data = {}
for key in keys: 
    data[key] = pd.read_csv(f"{dataset_dir}/smooth_{i}.csv")